In [2]:
#ignore warnings for clear printing
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression, LinearRegression, MultiTaskLasso,BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import seaborn as sns # More snazzy plotting library
import itertools
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier, MLPRegressor

##test results 
#  1.looks the absolute year number gives a good colleration for prediction. 
#  2. standarlistaion improves prediction by 3%
#  3. removing aspect ratio improves prediction by 2%


#open the file
#file_path =  "./working_dataset.csv"
file_path =  "../dataset/movie_metadata_budgets_utf8.csv"
dta = pd.read_csv(file_path)
dta_clean = dta
#remove the null values, that is fill NaN with there - FIXME: Rihards, naive implementation
dta_clean = dta_clean.fillna(value=0, axis=1)
dta_clean = dta_clean.dropna()
dta_clean = dta_clean.reindex_axis(sorted(dta_clean.columns), axis=1)


def mean (val, mean):
    if (val == 0) : return mean
    else: return val
dta_clean = dta_clean.reindex_axis(sorted(dta_clean.columns), axis=1)
#clean up data from zero rows 
for colname, colvalue in dta_clean.iteritems():
    if type(colvalue[1]) != str:
        dta_clean[colname] = dta_clean[colname].apply (lambda col: mean(col, dta_clean[colname].mean()))
dta_clean.count()

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


actor_1_facebook_likes       5043
actor_1_name                 5043
actor_2_facebook_likes       5043
actor_2_name                 5043
actor_3_facebook_likes       5043
actor_3_name                 5043
aspect_ratio                 5043
budget                       5043
cast_total_facebook_likes    5043
color                        5043
content_rating               5043
country                      5043
director_facebook_likes      5043
director_name                5043
duration                     5043
facenumber_in_poster         5043
genres                       5043
gross                        5043
imdb_score                   5043
language                     5043
movie_facebook_likes         5043
movie_imdb_link              5043
movie_title                  5043
num_critic_for_reviews       5043
num_user_for_reviews         5043
num_voted_users              5043
plot_keywords                5043
title_year                   5043
dtype: int64

In [25]:
#clasify the data for the logistic regression
def label_gross (gross):
    if (gross < 1000000) : return 1
    elif ((gross >= 1000000) & (gross < 10000000)) : return 2
    elif ((gross >= 10000000) & (gross < 50000000)) : return 3
    elif ((gross >= 50000000) & (gross < 200000000)) : return 4
    elif (gross >= 200000000) : return 5
    
dta_clean['gross_class'] = dta_clean.gross.apply (lambda gross: label_gross (gross))

dta_cl1 = dta_clean[dta_clean['gross_class'] == 1]
dta_cl2 = dta_clean[dta_clean['gross_class'] == 2]
dta_cl3 = dta_clean[dta_clean['gross_class'] == 3]
dta_cl4 = dta_clean[dta_clean['gross_class'] == 4]
dta_cl5 = dta_clean[dta_clean['gross_class'] == 5]

##build linear models for each class
y_1, X_1 = dmatrices('gross ~ budget + C(language) + cast_total_facebook_likes + \
                 director_facebook_likes + duration + facenumber_in_poster + imdb_score + \
                 movie_facebook_likes + num_critic_for_reviews + num_user_for_reviews + num_voted_users + title_year',
                 dta_cl5, return_type="dataframe")

#LR_1 = LinearRegression().fit(X_1, y_1)

##svr classificaiton
y, X = dmatrices('gross ~ budget + C(language) + cast_total_facebook_likes + \
                 director_facebook_likes + duration + facenumber_in_poster + imdb_score + \
                 movie_facebook_likes + num_critic_for_reviews + num_user_for_reviews + num_voted_users + title_year',
                 dta_clean, return_type="dataframe")
if True:
    for colname, colvalue in X.iteritems():
        nomalizer_scaler = preprocessing.Normalizer().fit(X[colname])
        X[colname] = nomalizer_scaler.transform(X[colname])[0]
if False:
    for colname, colvalue in X.iteritems():
        standard_scaler = preprocessing.StandardScaler().fit(X[colname])
        X[colname] = standard_scaler.transform(X[colname])[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
#model = SVR(kernel='rbf', verbose=2)
#model = LogisticRegression()
#model = DecisionTreeRegressor()
model = MLPRegressor(solver='lbfgs',  max_iter=20000)
model.fit(X_train, y_train)
# check the accuracy on the training set
#model.predict(X_test)
#LR_1.score(X_1, y_1)
model.score(X_test, y_test)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

0.26091057352519276